## Create Warehouse structure sql


### datetime
```sql

DROP TABLE IF EXISTS datetime_dimension;
CREATE TABLE datetime_dimension (
	"id" INTEGER PRIMARY KEY,
    "ds" timestamp,
    "year" SMALLINT,
    "month" SMALLINT,
    "day" SMALLINT,
    "week" SMALLINT,
    "hour" SMALLINT,
    "day_of_week" SMALLINT
);


INSERT INTO datetime_dimension
SELECT
	cast(to_char(ds, 'YYYYMMDDHH24') as INTEGER) as id,
	ds,
	EXTRACT(year from ds) as year,
	EXTRACT(month from ds) as month,
	EXTRACT(day from ds) as day,
	EXTRACT(WEEK from ds) as week,
	EXTRACT(HOUR from ds) as hour,
	EXTRACT(dow from ds) as day_of_week

from (
SELECT
	generate_series(
		(date '2020-01-01')::timestamp,
		(date '2021-01-31')::timestamp,
		interval '1 hour') as ds ) as t1
ON CONFLICT 
ON CONSTRAINT datetime_dimension_pkey DO NOTHING


```

## Products
```sql

DROP TABLE IF EXISTS product_dimension;
Create table product_dimension(
	id SERIAL PRIMARY KEY,
	product_id INTEGER NOT NULL UNIQUE,
	name VARCHAR(255),
    brand VARCHAR(255)
)

```

## Categories

```sql

DROP TABLE IF EXISTS categiry_dimension;
Create table categiry_dimension(
	id SERIAL PRIMARY KEY,
	category_id INTEGER NOT NULL UNIQUE,
	name VARCHAR(255),
)

```

## Channels

```sql

DROP TABLE IF EXISTS channel_dimension;
Create table channel_dimension(
	id SERIAL PRIMARY KEY,
	category_id INTEGER NOT NULL UNIQUE,
	name VARCHAR(255)
)

```


## Visits

```sql
DROP TABLE IF EXISTS visits;
CREATE TABLE visits (
  id serial PRIMARY KEY,
  datetime_id INTEGER NOT NULL UNIQUE,
  visitors_count INTEGER NOT NULL,
  CONSTRAINT fk_datetime FOREIGN KEY(datetime_id) REFERENCES datetime_dimension(id)
  )
```

## sales

```sql
DROP TABLE IF EXISTS sales;
CREATE TABLE sales (
  id serial PRIMARY KEY,
  datetime_id INTEGER NOT NULL,
  order_id INTEGER NOT NULL,
  product_id INTEGER NOT NULL,
  category_id INTEGER NOT NULL,
  channel_id INTEGER NOT NULL,
  total_sales INTEGER NOT NULL,
  CONSTRAINT fk_products FOREIGN KEY(product_id) REFERENCES product_dimension(id),
  CONSTRAINT fk_category FOREIGN KEY(category_id) REFERENCES category_dimension(id),
  CONSTRAINT fk_datetime FOREIGN KEY(datetime_id) REFERENCES datetime_dimension(id),
  CONSTRAINT fk_channel FOREIGN KEY(channel_id) REFERENCES channel_dimension(id)
  )
```

## Example sales insert
```sql
INSERT INTO sales (order_id, datetime_id, product_id, category_id, channel_id, total_sales) VALUES
	(
		10080,
		2021011200,
		(select id from product_dimension as p where p.product_id = 10),
		(select id from category_dimension as c where c.category_id = 1),
        (select id from channel_dimension as ch where ch.channel_id = 22),
		100
	)

;

```

## Extract sales data from source system

```sql
 SELECT
        oi.order_id,
        t.id as category_id,
        pv.product_id,
        o.channel_id,
        CAST(DATE_FORMAT(o.created_at, '%Y%m%d%H') as INTEGER) as datetime_dim,
        CAST(sum(oi.total) as INTEGER) as total
    FROM
        ecommerce_order_item oi
    JOIN 
        ecommerce_product_variant pv on pv.id = oi.variant_id
    JOIN 
        ecommerce_order o on oi.order_id = o.id
    JOIN
        ecommerce_channel ch on o.channel_id = ch.id
    JOIN
        ecommerce_product p on p.id = pv.product_id
    JOIN
        ecommerce_taxon t on t.id = p.main_taxon_id
    GROUP BY
        oi.order_id,
        t.id,
        pv.product_id,
        ch.id
```